## Read the dataset csv file

In [ ]:
!pip install accelerate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
df = pd.read_csv('reviews.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOEhZuqSqqWnaKRgv-9ABYdajFUB0WugPGh-SG-...,Eric Tie,https://play-lh.googleusercontent.com/a-/AOh14...,I cannot open the app anymore,1,0,5.4.0.6,2020-10-27 21:24:41,NaN,NaN,newest,com.anydo
1,gp:AOqpTOH0WP4IQKBZ2LrdNmFy_YmpPCVrV3diEU9KGm3...,john alpha,https://play-lh.googleusercontent.com/a-/AOh14...,I have been begging for a refund from this app...,1,0,NaN,2020-10-27 14:03:28,"Please note that from checking our records, yo...",2020-10-27 15:05:52,newest,com.anydo
2,gp:AOqpTOEMCkJB8Iq1p-r9dPwnSYadA5BkPWTf32Z1azu...,Sudhakar .S,https://play-lh.googleusercontent.com/a-/AOh14...,Very costly for the premium version (approx In...,1,0,NaN,2020-10-27 08:18:40,NaN,NaN,newest,com.anydo
3,gp:AOqpTOGFrUWuKGycpje8kszj3uwHN6tU_fd4gLVFy9z...,SKGflorida@bellsouth.net DAVID S,https://play-lh.googleusercontent.com/-75aK0WF...,"Used to keep me organized, but all the 2020 UP...",1,0,NaN,2020-10-26 13:28:07,What do you find troublesome about the update?...,2020-10-26 14:58:29,newest,com.anydo
4,gp:AOqpTOHls7DW8wmDFzTkHwxuqFkdNQtKHmO6Pt9jhZE...,Louann Stoker,https://play-lh.googleusercontent.com/-pBcY_Z-...,Dan Birthday Oct 28,1,0,5.6.0.7,2020-10-26 06:10:50,NaN,NaN,newest,com.anydo


In [5]:
# Separating positive and negative samples

# In[ ]:
def to_sentiment(rating):
    rating = int(rating)

    # Convert to class
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'


# Apply to the dataset
df['sentiment'] = df.score.apply(to_sentiment)

# Separating positive and negative samples
positive_samples = df[df['sentiment'] == 'positive'].sample(n=5000, random_state=42)
negative_samples = df[df['sentiment'] == 'negative'].sample(n=4800, random_state=42)
neutral_samples = df[df['sentiment'] == 'neutral'].sample(n=1990, random_state=42)
# Concatenating the samples to get the final subset
subset_df = pd.concat([positive_samples, negative_samples, neutral_samples])

# Shuffling the subset
subset_df = subset_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Displaying the first few rows of the subset
print(subset_df.head())

                                            reviewId        userName  \
0  gp:AOqpTOE1UysdX6xWAuBI02Hyjj__z3gqp_KH2OR18bw...        Trung Le   
1  gp:AOqpTOGXym7v6k2vyHR6-FkYVqf68bCGtKgxRKH1eps...      Ahmed Saad   
2  gp:AOqpTOHKr37K5JHJ8tCIFhg7Aqb7ncmAncAd2FWtkAK...     Sarah Yaker   
3  gp:AOqpTOGoX-i60DkbvmvIqE-qd8Ee_9wFkSwuLjZn1Pl...   A Google user   
4  gp:AOqpTOGymqdRS7R2G2VlVaDJsmzHW1SWBJVZujNOeSH...  Vernon Hedrick   

                                           userImage  \
0  https://play-lh.googleusercontent.com/a-/AOh14...   
1  https://play-lh.googleusercontent.com/--vf_H0N...   
2  https://play-lh.googleusercontent.com/a-/AOh14...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/a-/AOh14...   

                                             content  score  thumbsUpCount  \
0  Turning your life into a RPG is a great way to...      4              0   
1   No dark theme without a premium subscription? Ok      2              0

In [6]:


# Getting the total number of samples
total_samples = subset_df.shape[0]

# Counting the number of positive and negative samples
class_counts = subset_df['sentiment'].value_counts()

# Displaying the results
print(f'Total number of samples: {total_samples}')
print(f'Number of positive samples: {class_counts["positive"]}')
print(f'Number of negative samples: {class_counts["negative"]}')
print(f'Number of neutral samples: {class_counts["neutral"]}')

Total number of samples: 11790
Number of positive samples: 5000
Number of negative samples: 4800
Number of neutral samples: 1990


## Process the data

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
def process_data(row):

    text = row['content']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    # Updated for 3-class classification: Assuming 0:negative, 1:neutral, 2:positive
    # Note: Your current 'subset_df' only contains 'positive' and 'negative' samples.
    # For true 3-class classification, you would need to adjust your data loading and sampling
    # to include a 'neutral' class from your dataset, if available.
    if row['sentiment'] == 'negative':
        label = 0
    elif row['sentiment'] == 'neutral': # Placeholder for a 'neutral' sentiment class
        label = 1
    elif row['sentiment'] == 'positive':
        label = 2
    else:
        # Default to 0 or handle other cases if sentiment values are unexpected
        label = 0 # Consider how to handle unknown sentiments in your dataset

    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [10]:
print(process_data({
    'content': 'this is a sample review of a movie.',
    'sentiment': 'positive'
}))

{'input_ids': [101, 2023, 2003, 1037, 7099, 3319, 1997, 1037, 3185, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'label': 2, 'text': 'this is a sample review of a movie.'}


In [12]:
processed_data = []

for i in range(len(subset_df)):
    processed_data.append(process_data(subset_df.iloc[i]))

## Generate the dataset

In [13]:
from sklearn.model_selection import train_test_split

new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)

In [14]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

## Create a model

In [15]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=3 # Changed from 2 to 3 for 3-class classification
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments
import torch
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


from sklearn.metrics import f1_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"f1": f1_score(labels, predictions, average='weighted')} # Added average='weighted' for multi-class F1


from transformers import TrainerCallback

class F1ScoreCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.best_f1 = 0.0
        self.best_model = None

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics["eval_f1"] > self.best_f1:
            self.best_f1 = metrics["eval_f1"]
            self.best_model = kwargs["model"].to("cpu")
            # Saving the best model
            self.best_model.save_pretrained(f"./best_model_epoch_{state.epoch}")



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


from transformers import Trainer

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        # Move the model to the GPU
        model.to(self.args.device)

        # Move all tensors in inputs to the GPU device
        inputs = {k: v.to(self.args.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        return super().training_step(model, inputs)




# Initializing the Custom Trainer with the same parameters as before
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    compute_metrics=compute_metrics,
    callbacks=[F1ScoreCallback()]
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

## Train and Evaluate the model

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,No log,0.308497,0.873799
2,No log,0.310922,0.873371
3,0.260800,0.372663,0.879316


TrainOutput(global_step=564, training_loss=0.24384238787576662, metrics={'train_runtime': 409.0993, 'train_samples_per_second': 87.998, 'train_steps_per_second': 1.379, 'total_flos': 1192206587904000.0, 'train_loss': 0.24384238787576662, 'epoch': 3.0})

In [ ]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        # Move the model to the GPU
        model.to(self.args.device)

        # Move all tensors in inputs to the GPU device
        inputs = {k: v.to(self.args.device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
        return super().training_step(model, inputs)




# Initialize the Custom Trainer with the same parameters as before
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    compute_metrics=compute_metrics,
    callbacks=[F1ScoreCallback()]
)
trainer.evaluate()

{'eval_loss': 0.372662752866745,
 'eval_f1': 0.8793160144689246,
 'eval_runtime': 9.8727,
 'eval_samples_per_second': 303.869,
 'eval_steps_per_second': 4.761}

## Save the model

In [ ]:
model.save_pretrained('./model/')

## Load the model

In [ ]:
from transformers import AutoModelForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [ ]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

## Get predictions

In [ ]:
import torch
import numpy as np

def get_prediction(text, model, tokenizer):
    # Encoding the input text
    encoding = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    # Moving the input encoding to the same device as the model
    encoding = {k: v.to(model.device) for k, v in encoding.items()}

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits

    # Converting logits to probabilities using softmax for multi-class
    softmax = torch.nn.Softmax(dim=1) # Apply softmax across the class dimension
    probs = softmax(logits) # Apply softmax to the logits
    probs = probs.squeeze().cpu().detach().numpy() # Squeeze and move to numpy array

    # Determining the label using the highest probability
    predicted_label_idx = np.argmax(probs)

    # Map index to sentiment string: Assuming 0:negative, 1:neutral, 2:positive
    sentiment_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    predicted_sentiment = sentiment_map.get(predicted_label_idx, 'Unknown') # Handle unexpected indices

    return {
        'sentiment': predicted_sentiment,
        'probability': probs[predicted_label_idx] # Probability of the predicted sentiment
    }

text_to_predict = "This is a positive example."
prediction = get_prediction(text_to_predict, new_model, new_tokenizer)
print(prediction)


{'sentiment': 'Positive', 'probability': 0.56779325}


In [ ]:
get_prediction('The movie was awful.', new_model, new_tokenizer)

{'sentiment': 'Negative', 'probability': 0.9465001}